In [2]:
begin
	import Pkg
	Pkg.activate("..")
	Pkg.instantiate()
	
    push!(LOAD_PATH, "$(@__DIR__)/../src")
    using Test
	using NoisyReach
    using QuadGK
    using ControlSystemsBase
    using LinearAlgebra
    using ReachabilityAnalysis
end

  Activating project at `c:\My Project\NoisyReach.jl`


In [3]:
const Ts = 0.01 # Sampling period

0.01

In [4]:
const Dc = 0.005 # sensor-to-actuator delay

0.005

In [5]:
# Example from page 33 of lecture 17 (Ts = 0.01, Dc = 0.005)
# Expected:
# ϕ = [1 0.001; -0.001 0.999]
# Γ₁ = [0; 0.0005]
# Γ₀ = [0; 0.0005]
#
# ϕ_aug = [1 0.001 0; -0.001 0.999 0.0005; 0 0 0]
# Γ_aug = [0 0.0005 1]
# C_aug = [1 0 0]
sys = let
    A = [0 1; -1 -1]
    B = [0; 1]
    C = [1 0]
    D = 0
    ss(A, B, C, D)
end

StateSpace{Continuous, Int64}
A = 
  0   1
 -1  -1
B = 
 0
 1
C = 
 1  0
D = 
 0

Continuous-time state-space model

In [6]:
sys_aug_ = let
    ϕ = ℯ^(Ts * sys.A)
    Γ₁ = matrix_integral(A, B, Dc, Ts)
    Γ₀ = matrix_integral(A, B, 0.0, Ts - Dc)
    ϕ_aug = [ϕ Γ₁; 0 0 0]
    Γ_aug = [Γ₀; I]
    C_aug = [sys.C 0]
    ss(ϕ_aug, Γ_aug, C_aug, sys.D, Ts)
end

UndefVarError: UndefVarError: `matrix_integral` not defined

In [7]:
#sys_aug = c2d(sys, Ts) * delay(Dc, Ts)
sys_aug = c2d(sys, Ts) * thiran(Dc, Ts)

StateSpace{Discrete{Float64}, Float64}
A = 
  0.9999501666658346    0.009950000415833335   4.4296297035802466e-5
 -0.009950000415833335  0.9900001662500014     0.008844444814074077
  0.0                   0.0                   -0.3333333333333333
B = 
 1.6611111388425926e-5
 0.0033166668052777787
 1.0
C = 
 1.0  0.0  0.0
D = 
 0.0

Sample Time: 0.01 (seconds)
Discrete-time state-space model

In [8]:
K = lqr(ControlSystemsBase.Discrete, sys_aug.A, sys_aug.B, I, I)

1×3 Matrix{Float64}:
 0.239321  0.418528  -0.167879

In [9]:
function matrix_integral(A::AbstractMatrix, B::AbstractMatrix, a::Float64, b::Float64)
    result, _ = quadgk(s ->  exp(A * s) * B, a, b)
    return result
end

matrix_integral (generic function with 1 method)

In [10]:
sys = benchmarks[:F1]
const period = 0.02
const Dc1 = 0.005
const Dc2 = 0.015
const errors = [0, 0]

2-element Vector{Int64}:
 0
 0

In [15]:
sys_aug = let
    ϕ = ℯ^(period * sys.A)
    Γ2 = matrix_integral(sys.A, sys.B, Dc2, period)
    Γ1 = matrix_integral(sys.A, sys.B, Dc1, Dc2)
    Γ3 = matrix_integral(sys.A, sys.B, 0.0, Dc1)
    ϕ_aug = [ϕ Γ3; 0 0 0]
    Γ_aug = [Γ1 Γ2; 0 I]
    C_aug = [sys.C 0]
    D_aug = [sys.D 0]
    ss(ϕ_aug, Γ_aug, C_aug, D_aug, Ts)
end
K = lqr(ControlSystemsBase.Discrete, sys_aug.A, sys_aug.B, I, I)
Φ = sys_aug.A + sys_aug.B * K

3×3 Matrix{Float64}:
 1.0127    0.149407  0.00336739
 0.172671  1.26289   0.122367
 0.307036  0.48328   0.0441294

In [27]:
λ11 = 0.3
λ12 = 0.25
λ21 = 0.1
λ22 = 0.1
K_error = [K[1,:][1]*(1+λ11) K[1,:][2]*(1+λ12) K[1,:][3]; K[2,:][1]*(1+λ21) K[2,:][2]*(1+λ22) K[2,:][3]]

0.09956091952339212

In [13]:
E = Zonotope(zeros(Float64, 2), Diagonal(errors))
W = get_error_bound(sys_aug.B, K, E)
x0center = 10.
x0size = 1.
x0 = Zonotope(x0center * ones(2), x0size * I(2))
r = reach(Φ, x0, W, 100)

MethodError: MethodError: no method matching Zonotope(::Vector{Float64}, ::Diagonal{Int64, Vector{Int64}})

Closest candidates are:
  Zonotope(::VN, !Matched::MN) where {N, VN<:AbstractVector{N}, MN<:AbstractMatrix{N}}
   @ LazySets C:\Users\Tingan Zhu\.julia\packages\LazySets\qUfTe\src\Sets\Zonotope\Zonotope.jl:94
  Zonotope(::VN, !Matched::AbstractVector{VN}) where VN<:(AbstractVector)
   @ LazySets C:\Users\Tingan Zhu\.julia\packages\LazySets\qUfTe\src\Sets\Zonotope\Zonotope.jl:106
